# Especialização em Inteligência Artificial

**Aprendizado de Máquina - Aula 3.4: Algoritmos de classificação**

Código de exemplo desenvolvido pelo docente [Adriano Rivolli](mailto:rivolli@utpfr.edu.br)

*O código apresenta o uso de diferentes algoritmos de classificação*

In [1]:
# Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, f1_score, accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

In [2]:
# Carregar o dataset spambase
spambase = fetch_openml(name='spambase', version=1, as_frame=True)
X = spambase.data
y = spambase.target

# Converter o target de string para int, se necessário
y = y.astype(int)

In [4]:
# Algoritmos a serem comparados
knn = KNeighborsClassifier()
lr = LogisticRegression(random_state=21, max_iter=3000)
dt = DecisionTreeClassifier(random_state=0)
svm = SVC(random_state=0)

print("Comparação da medida F1 (validação cruzada de 10-folds):")
print("-" * 60)

# k-NN
scores_knn = cross_val_score(knn, X, y, cv=10, scoring='f1_macro')
print(f"F1-Score médio do k-NN: {scores_knn.mean():.4f}")

# Regressão Logística
scores_lr = cross_val_score(lr, X, y, cv=10, scoring='f1_macro')
print(f"F1-Score médio da Regressão Logística: {scores_lr.mean():.4f}")

# Árvore de Decisão
scores_dt = cross_val_score(dt, X, y, cv=10, scoring='f1_macro')
print(f"F1-Score médio da Árvore de Decisão: {scores_dt.mean():.4f}")

# SVM
scores_svm = cross_val_score(svm, X, y, cv=10, scoring='f1_macro')
print(f"F1-Score médio do SVM: {scores_svm.mean():.4f}")

Comparação da medida F1 (validação cruzada de 10-folds):
------------------------------------------------------------
F1-Score médio do k-NN: 0.7768
F1-Score médio da Regressão Logística: 0.9144
F1-Score médio da Árvore de Decisão: 0.8932
F1-Score médio do SVM: 0.6696


In [5]:
# Testando K-NN com diferentes valores de n_neighbors
k_values = [3, 5, 7, 9]

print("\nAvaliação do K-NN com diferentes valores de k (validação cruzada de 10-folds):")
print("-" * 80)

for k in k_values:
    knn_k = KNeighborsClassifier(n_neighbors=k)

    # Previsões para o cálculo do relatório de classificação
    y_pred_knn_k = cross_val_predict(knn_k, X, y, cv=10)

    # Gerar o relatório de classificação
    report = classification_report(y, y_pred_knn_k, output_dict=True)

    # Extrair precisão e revocação para a classe 1 (spam)
    precision_spam = report['1']['precision']
    recall_spam = report['1']['recall']

    print(f"Para k = {k}:")
    print(f"  Precisão (spam): {precision_spam:.4f}")
    print(f"  Revocação (spam): {recall_spam:.4f}")


Avaliação do K-NN com diferentes valores de k (validação cruzada de 10-folds):
--------------------------------------------------------------------------------
Para k = 3:
  Precisão (spam): 0.7288
  Revocação (spam): 0.7204
Para k = 5:
  Precisão (spam): 0.7399
  Revocação (spam): 0.7137
Para k = 7:
  Precisão (spam): 0.7217
  Revocação (spam): 0.7066
Para k = 9:
  Precisão (spam): 0.7103
  Revocação (spam): 0.6966


In [6]:
# Acurácia da Árvore de Decisão com max_depth=5
dt_depth5 = DecisionTreeClassifier(max_depth=5, random_state=0)

# Validação cruzada com 10-folds
accuracy_dt = cross_val_score(dt_depth5, X, y, cv=10, scoring='accuracy')

print("\nCálculo da Acurácia da Árvore de Decisão (max_depth=5):")
print("-" * 60)
print(f"Acurácia média: {accuracy_dt.mean():.4f}")


Cálculo da Acurácia da Árvore de Decisão (max_depth=5):
------------------------------------------------------------
Acurácia média: 0.9031


In [7]:
# Medida AUC para o SVM
svm_auc = SVC(C=0.1, gamma=0.1, random_state=0, probability=True)

# Validação cruzada com 10-folds para o cálculo do AUC
# 'auc' é um alias para 'roc_auc_score'
auc_scores = cross_val_score(svm_auc, X, y, cv=10, scoring='roc_auc')

print("\nCálculo da medida AUC para o SVM (C=0.1, gamma=0.1):")
print("-" * 60)
print(f"AUC médio: {auc_scores.mean():.4f}")


Cálculo da medida AUC para o SVM (C=0.1, gamma=0.1):
------------------------------------------------------------
AUC médio: 0.8579


In [8]:
# Usando o dataset inteiro para Regressão Logística
lr_full = LogisticRegression(random_state=21, max_iter=3000)

# Treinar o modelo com o dataset completo
lr_full.fit(X, y)

# Acessar os coeficientes e os nomes das features
coefficients = lr_full.coef_[0]
feature_names = spambase.feature_names

# Criar um DataFrame para facilitar a visualização
coef_df = pd.DataFrame({
    'Atributo': feature_names,
    'Coeficiente': coefficients,
    'Coeficiente_Absoluto': np.abs(coefficients)
})

# Encontrar o atributo com o maior valor de coeficiente absoluto
most_important_attribute = coef_df.loc[coef_df['Coeficiente_Absoluto'].idxmax()]

print("\nIdentificação do atributo mais importante (Regressão Logística):")
print("-" * 60)
print("O atributo mais importante é '", most_important_attribute['Atributo'], "' com um valor de coeficiente absoluto de", f"{most_important_attribute['Coeficiente_Absoluto']:.4f}.")


Identificação do atributo mais importante (Regressão Logística):
------------------------------------------------------------
O atributo mais importante é ' word_freq_george ' com um valor de coeficiente absoluto de 3.9741.
